In [1]:
%load_ext autoreload
%autoreload 2
from bedoner.lang.juman import Japanese

In [8]:
from spacy.strings import StringStore
from spacy.vocab import Vocab

with open("../bedore-nlp-modules/data/Japanese_L-12_H-768_A-12_E-30_BPE/vocab.txt") as f:
    vs = []
    for line in f:
        vs.append(line[:-1])

s=StringStore(vs)

v=Vocab(strings=s)
v.to_disk("data/vocab.Japanese_L-12_H-768_A-12_E-30_BPE")

In [9]:
import mojimoji

In [10]:
from spacy_pytorch_transformers.pipeline.wordpiecer import PyTT_WordPiecer 

In [5]:
from spacy_pytorch_transformers._tokenizers import SerializableBertTokenizer

In [27]:
nlp = Japanese(v)
w = PyTT_WordPiecer(v)
wp = SerializableBertTokenizer("../bedore-nlp-modules/data/Japanese_L-12_H-768_A-12_E-30_BPE/vocab.txt", do_lower_case=False, tokenize_chinese_chars=False)
w.model = wp
nlp.add_pipe(w)

In [28]:
text="EXILEのATSUSHIと中島美嘉が14日ニューヨーク入り"
doc=nlp(text)

In [47]:
def model_fn_builder(
    bert_config, num_labels, init_checkpoint, use_one_hot_embeddings, max_seq_length
):
    """Returns `model_fn` closure for TPUEstimator."""

    def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
        """The `model_fn` for TPUEstimator."""

        tf.logging.info("*** Features ***")
        for name in sorted(features.keys()):
            tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

        input_ids = features["input_ids"]
        input_mask = features["input_mask"]
        segment_ids = features["segment_ids"]
        label_ids = features["label_ids"]

        is_training = mode == tf.estimator.ModeKeys.TRAIN

        (total_loss, per_example_loss, logits, predicts) = create_model(
            bert_config,
            is_training,
            input_ids,
            input_mask,
            segment_ids,
            label_ids,
            num_labels,
            use_one_hot_embeddings,
            max_seq_length,
        )

        tvars = tf.trainable_variables()
        initialized_variable_names = {}
        # scaffold_fn = None
        if init_checkpoint:
            (
                assignment_map,
                initialized_variable_names,
            ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)

        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

        output_spec = None
        if mode == tf.estimator.ModeKeys.PREDICT:
            output_spec = tf.estimator.EstimatorSpec(
                mode=mode, predictions=predicts  # {"probabilities": probabilities},
            )
        return output_spec

    return model_fn

In [50]:
from bedore_nlp_modules.bert_modules import modeling

In [53]:
from pathlib import Path

In [54]:
__dir__ = Path()

In [246]:
wp

In [248]:
w.model

In [250]:
nlp.pipe_names

['pytt_wordpiecer']

In [242]:
bert_config = modeling.BertConfig.from_json_file(str(__dir__/"data/Japanese_L-12_H-768_A-12_E-30_BPE/bert_config.json"))

In [62]:
import pickle

In [64]:
with open(str(__dir__ / "data/bert_result_ene_0/label2id.pkl"), "rb") as f:
    label2id = pickle.load(f)

In [66]:
num_labels=len(label2id) + 1

In [182]:
bert_dir = __dir__ / "data/Japanese_L-12_H-768_A-12_E-30_BPE/"
model_dir = __dir__ / "data/bert_result_ene_0/"

In [183]:
init_checkpoint = str(bert_dir/ "bert_model.ckpt")

In [184]:
max_seq_length=128
predict_batch_size=8
drop_remainder=False

In [185]:
model_fn = model_fn_builder(
    bert_config=bert_config,
    num_labels=num_labels,
    init_checkpoint=init_checkpoint,
    use_one_hot_embeddings=None,
    max_seq_length=max_seq_length,
)

In [252]:
import json

In [254]:
with open("data/bert_result_ene_0/id2label.json", "w") as f:
    json.dump(id2label, f)

In [186]:
run_config = tf.estimator.RunConfig(model_dir=model_dir)

In [187]:
estimator = tf.estimator.Estimator(
    model_fn=model_fn, config=run_config, params={"batch_size": 10}
)

In [188]:
features = {}

In [189]:
def pad(a, l, c= 0):
    if len(a) >= l:
        return a[:l]
    return a + [c]*(l - len(a))

In [190]:
docs = [doc]

In [191]:
import numpy as np

In [192]:
import itertools as it

In [193]:
features["input_ids"] = np.array([pad(doc._.pytt_word_pieces, max_seq_length) for doc in docs])

In [194]:
features["input_mask"] = np.array([pad([1 for _ in range(len(doc._.pytt_word_pieces))], max_seq_length) for doc in docs])

In [195]:
segs = []
for doc in docs:
    seg = []
    for i,s in enumerate(doc._.pytt_segments):
        seg += [i] * len(s._.pytt_word_pieces)
    segs.append(pad(seg, max_seq_length))
features["segment_ids"] = np.array(segs)

In [196]:
doc._.pytt_alignment

[[1], [2], [3, 4, 5], [6], [7], [8, 9], [10], [11], [12], [13], [14]]

In [197]:
label_ids = []

In [198]:
for doc in docs:
    label_id = [label2id["[CLS]"]]
    O,X = label2id["O"], label2id["X"]
    for a in doc._.pytt_alignment:
        label_id + [O] + [X] * (len(a)-1)
    label_ids.append(pad(label_id, max_seq_length))

In [199]:
features["label_ids"] = np.array(label_ids)

In [200]:
f=tf.estimator.inputs.numpy_input_fn(features, shuffle=False)

In [215]:
_res = list(estimator.predict(f))

In [216]:
id2label = {v:k for k,v in label2id.items()}

In [223]:
res = list(id2label[r] for r in _res[0] if r != 0f)

In [278]:
with open("data/bert_result_ene_0/id2label.pkl", "wb") as f:
    pickle.dump(id2label,f)

In [267]:
_res

[array([406, 334, 404, 267, 405, 405, 404, 267, 266, 405, 404,  78,  79,
         46, 404, 407,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])]

In [234]:
from spacy.tokens import Span

In [238]:
new_ents = []
cur = []
for i,a in enumerate(doc._.pytt_alignment):
    l = res[a[0]]
    doc[i].ent_type_ = l
    if l.startswith("B-"):
        if cur:
            new_ents.append(Span(doc,cur[0], cur[1], cur[2]))
        cur = [i,i+1,l]
    elif l.startswith("I-"):
        cur[1]+=1
if cur:
    new_ents.append(Span(doc,cur[0], cur[1], cur[2]))
doc.ents = new_ents

In [262]:
ents=[]
for e in doc.ents:
    ents.append((e.text, e.label_[2:]))

In [263]:
ents

[('ＥＸＩＬＥ', 'Show_Organization'),
 ('ＡＴＳＵＳＨＩ', 'Person'),
 ('中島美嘉', 'Person'),
 ('１４日', 'Date'),
 ('ニューヨーク', 'City')]

In [ ]:
nlp.get_pipe[""]

In [266]:
nlp.get_pipe("pytt_wordpiecer").model